In [1]:
'''
Author: Ngawang Gurung
Date: 2024/07/24
'''

'\nAuthor: Ngawang Gurung\nDate: 2024/07/24\n'

In [1]:
import pandas as pd
from datetime import datetime
import time
import random
import os
from dotenv import load_dotenv

from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import base64, expr
from helper import table_df

In [2]:
spark = SparkSession.builder \
    .appName("Spark") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/24 17:06:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/24 17:06:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
pandas_df = table_df('client_rw', 'fc_account_master')
pandas_df.head()

/home/user/Projects/eXtensoData/Kafka/Kafka_AES/helper.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=con.connection)


,account_number,customer_code,product,product_category,acc_open_date,acc_closed_date,active_flag
0,02XYZXYZ10015592101,KL0255921,SBA,SBPPS,1999-03-10,None,0
1,02XYZXYZ10015593701,KL0255937,SBA,SBPPS,1999-03-10,None,0
2,02XYZXYZ10015593801,KL0255938,SBA,SBPPS,1999-03-10,None,0
3,02XYZXYZ10015594801,KL0255948,SBA,SBANU,1999-03-22,None,0
4,02XYZXYZ10015597601,KL0255976,SBA,SBPPS,1999-04-22,None,0


In [17]:
pandas_df.fillna(-1, inplace=True)

/tmp/ipykernel_40783/4270938028.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pandas_df.fillna(-1, inplace=True)


In [19]:
df = spark.createDataFrame(pandas_df)
df.show()

+--------------------+-------------+-------+----------------+-------------+---------------+-----------+
|      account_number|customer_code|product|product_category|acc_open_date|acc_closed_date|active_flag|
+--------------------+-------------+-------+----------------+-------------+---------------+-----------+
| 02XYZXYZ10015592101|    KL0255921|    SBA|           SBPPS|   1999-03-10|             -1|          0|
| 02XYZXYZ10015593701|    KL0255937|    SBA|           SBPPS|   1999-03-10|             -1|          0|
| 02XYZXYZ10015593801|    KL0255938|    SBA|           SBPPS|   1999-03-10|             -1|          0|
| 02XYZXYZ10015594801|    KL0255948|    SBA|           SBANU|   1999-03-22|             -1|          0|
| 02XYZXYZ10015597601|    KL0255976|    SBA|           SBPPS|   1999-04-22|             -1|          0|
| 02XYZXYZ10015597701|    KL0255977|    SBA|           SBPPS|   1999-04-22|             -1|          0|
| 02XYZXYZ10015597801|    KL0255978|    SBA|           SBPPS|   

In [20]:
load_dotenv()

SUPER_SECRET_KEY = os.getenv('SUPER_SECRET_KEY') # Super secret key (must be 16, 24, or 32 bytes long)
df_encrypted = df.withColumn('account_number', base64(expr(f"aes_encrypt(account_number, '{SUPER_SECRET_KEY}', 'ECB')")))

In [21]:
df_encrypted.show(5)

+--------------------+-------------+-------+----------------+-------------+---------------+-----------+
|      account_number|customer_code|product|product_category|acc_open_date|acc_closed_date|active_flag|
+--------------------+-------------+-------+----------------+-------------+---------------+-----------+
|+Y7DNNNCRLxa9tHil...|    KL0255921|    SBA|           SBPPS|   1999-03-10|             -1|          0|
|U7RRmweke5W8oVRNv...|    KL0255937|    SBA|           SBPPS|   1999-03-10|             -1|          0|
|U7RRmweke5W8oVRNv...|    KL0255938|    SBA|           SBPPS|   1999-03-10|             -1|          0|
|Khed5di6hfdejAOOB...|    KL0255948|    SBA|           SBANU|   1999-03-22|             -1|          0|
|wt0lfnWsGTalCHEtZ...|    KL0255976|    SBA|           SBPPS|   1999-04-22|             -1|          0|
+--------------------+-------------+-------+----------------+-------------+---------------+-----------+
only showing top 5 rows



In [22]:
# Convert each Row to a dictionary
df_list = [row.asDict() for row in df_encrypted.collect()]

In [23]:
KAFKA_TOPIC = "am"
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

producer = KafkaProducer(bootstrap_servers = KAFKA_BOOTSTRAP_SERVERS, value_serializer = lambda x: x.encode('utf-8'))

In [24]:
print("Kafka Producer Application Started ... ")

for message in df_list:
    message_data = ','.join(str(message[column]) for column in df.columns)
    print("Message Type:", type(message_data))
    print("Message:", message_data)
    producer.send(KAFKA_TOPIC, message_data)
    time.sleep(1)

print("Kafka Producer Application Completed.")

Kafka Producer Application Started ... 
Message Type: <class 'str'>
Message: +Y7DNNNCRLxa9tHiluVGkYY5B2W/n3ZpLplkJINLIdM=,KL0255921,SBA,SBPPS,1999-03-10,-1,0
Message Type: <class 'str'>
Message: U7RRmweke5W8oVRNvaKXpZJPf6MTWIf+3pVCi/uDj3w=,KL0255937,SBA,SBPPS,1999-03-10,-1,0
Message Type: <class 'str'>
Message: U7RRmweke5W8oVRNvaKXpW7opsj+z4fEE98TMsDU5ZQ=,KL0255938,SBA,SBPPS,1999-03-10,-1,0
Message Type: <class 'str'>
Message: Khed5di6hfdejAOOBP6XBG7opsj+z4fEE98TMsDU5ZQ=,KL0255948,SBA,SBANU,1999-03-22,-1,0
Message Type: <class 'str'>
Message: wt0lfnWsGTalCHEtZvy671U78SwdSDGmCdZQ2GwRpAw=,KL0255976,SBA,SBPPS,1999-04-22,-1,0
Message Type: <class 'str'>
Message: wt0lfnWsGTalCHEtZvy675JPf6MTWIf+3pVCi/uDj3w=,KL0255977,SBA,SBPPS,1999-04-22,-1,0
Message Type: <class 'str'>
Message: wt0lfnWsGTalCHEtZvy6727opsj+z4fEE98TMsDU5ZQ=,KL0255978,SBA,SBPPS,1999-04-22,-1,0
Message Type: <class 'str'>
Message: FtDcN60NxCHnK87AgC9WyxWfZKBFMtej55gj9Qs8d94=,KL0255983,SBA,SBPPS,1999-04-22,-1,0
Message Type: <c